In [1]:
import numpy as np
import sys
import ipdb

sys.path.append("../")

sys.path.append("../utils/")
from utils_functions import *

from dd_game import *
from two_stage_player import *

# Configurations

In [2]:
#TODO: There are many configuration parameters here. It is important to understand the effect of all.
d = 2
coeff_var = 1

Sigma_x_p1 = np.eye(d)
Sigma_x_p2 = np.eye(d)

sigma_y_p1 = 10
sigma_y_p2 = 10

beta_p1 = np.random.normal(size = d)
beta_p2 = np.random.normal(size = d)

mu_p1 = np.random.normal(scale = coeff_var, size = d)
mu_p2 = np.random.normal(scale = coeff_var, size = d)
gamma_p1 = np.random.normal(scale = coeff_var, size = d)
gamma_p2 = np.random.normal(scale = coeff_var, size = d)

In [3]:
p1_data_params = (Sigma_x_p1, sigma_y_p1, beta_p1, mu_p1, gamma_p1)
p1_data_generating_func = sample_from_location_family

p2_data_params = (Sigma_x_p2, sigma_y_p2, beta_p2, mu_p2, gamma_p2)
p2_data_generating_func = sample_from_location_family

num_rounds = 1000
num_alternate_rounds = 1000
num_test = 10000

In [4]:
#Initialize game with configurations specified above
player_two = TwoStagePlayer()

player_one = TwoStagePlayer()

game = DecisionDependentGame(player_one, player_two, p1_data_params, p2_data_params, 
                             p1_data_generating_func, p2_data_generating_func, 
                             num_rounds, num_alternate_rounds, num_test)

# Run Game

In [5]:
game.run_train()

Stage 1 finished. Performing estimation now
Stage 1 finished. Performing estimation now
Stage 2 finished. Finding qs now.
Stage 2 finished. Finding qs now.


(array([ 0.9057997, -1.1684379]), array([-1.57324929, -0.75548668]))

In [18]:
game.evaluate_test_perf_risk_p1()

118.05207915364169

In [19]:
game.evaluate_test_perf_risk_p2()

117.73550499598703

In [10]:
player_one.theta_history[-1]
player_two.theta_history[-1]

array([-1.57324929, -0.75548668])

In [21]:
player_one.mu_hat
#player_two.gamma_hat

array([ 1.10670263, -0.64279062])

In [22]:
mu_p1

array([ 0.58262018, -0.85886999])